In [1]:
import numpy as np
import os
import cv2
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32, kernel_size=2,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=2,padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=2,padding=1)
        self.fc1 = nn.Linear(128, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = F.relu(F.max_pool2d(self.conv3(x),7))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net()

In [3]:
path_to_dataset = '/Users/beene/Documents/BMM-2023/SN_mnist/training' #ПУТЬ ДО ДАТАСЕТА

In [4]:
os.listdir('training')

['.DS_Store', '9', '0', '7', '6', '1', '8', '4', '3', '2', '5']

In [5]:
data = []
numbers = os.listdir('training')
for number in numbers:
    if number!=".DS_Store":
        names = os.listdir('training/' + number)[:1000]
        for name in names:
            im_frame = Image.open(f'training/{number}/{name}')
            np_frame = np.array(im_frame.getdata()).reshape(1,28,28)
            data.append(np_frame)
        
data = np.array(data)

In [6]:
class Dataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data/255).type(torch.float)
        
        labels = []
        for i in range(10):
            for j in range(int(len(data)/10)):
                labels.append(i)
        self.labels = labels
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [7]:
dataset = Dataset(data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 32)

In [8]:
samples, labels = next(iter(dataloader))

print('Samples shape: ', samples.shape)
print('Labels shape: ', labels.shape)

Samples shape:  torch.Size([32, 1, 28, 28])
Labels shape:  torch.Size([32])


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0009)

In [14]:
for epoch in range(20):
    running_loss = 0.0
    for samples, labels in dataloader:
        pred = net(samples)
        loss = criterion(pred, labels)
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
    print(f'Epoch: {epoch+1}, loss: {loss}')

Epoch: 1, loss: 0.0010479817865416408
Epoch: 2, loss: 0.07754584401845932
Epoch: 3, loss: 0.1697636842727661
Epoch: 4, loss: 0.11088471859693527
Epoch: 5, loss: 0.0074396999552845955
Epoch: 6, loss: 0.10072565078735352
Epoch: 7, loss: 0.026062382385134697
Epoch: 8, loss: 0.00031744575244374573
Epoch: 9, loss: 0.009801927953958511
Epoch: 10, loss: 0.00036245083902031183
Epoch: 11, loss: 0.014585684984922409
Epoch: 12, loss: 8.526111196260899e-05
Epoch: 13, loss: 0.00045716436579823494
Epoch: 14, loss: 0.0003709282318595797
Epoch: 15, loss: 0.003158816136419773
Epoch: 16, loss: 7.282865408342332e-05
Epoch: 17, loss: 0.00010599569941405207
Epoch: 18, loss: 0.004436803050339222
Epoch: 19, loss: 2.2343150703818537e-05
Epoch: 20, loss: 0.0010018125176429749


In [18]:
data = []
numbers = os.listdir('training')
for number in numbers:
    if number!=".DS_Store":
        names = os.listdir('training/' + number)[1000:2000]
        for name in names:
            im_frame = Image.open(f'training/{number}/{name}')
            np_frame = np.array(im_frame.getdata()).reshape(1,28,28)
            data.append(np_frame)
        
data = np.array(data)

In [19]:
dataset = Dataset(data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 32)

In [20]:
cnt=0
for sample, label in dataloader:
    pred = net(sample)
    _, v = torch.max(pred, 1)
    l=torch.sum(v == label)
    cnt+=l
print(cnt/len(dataset))


    

tensor(1.)


In [21]:
10000/3.6

2777.777777777778